# Amazon SageMaker XGBoost Bring Your Own Model

_**미리 학습된 xgboost 모델을 호스팅하기 (레퍼런스: https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/xgboost_bring_your_own_model/xgboost_bring_your_own_model.ipynb)**_

---

---

## 목차

1. [배경](#배경)
1. [설정](#설정)
1. [XGBoost model 학습](#XGBoost-model-학습)
1. [미리 학습된 모델을 S3에 업로드](#미리-학습된-모델을-S3에-업로드)
1. [모델에 대한 호스팅 설정](#모델에-대한-호스팅-설정)
1. [사용할 모델의 유효성 검사](#사용할-모델의-유효성-검사)

---
## 배경

Amazon SageMaker에는 호스팅된 노트북 환경, 분산, 서버리스 교육 및 실시간 호스팅을 지원하는 기능이 포함되어 있습니다.이러한 세 가지 서비스를 모두 함께 사용할 때 가장 효과적이라고 생각하지만 독립적으로 사용할 수도 있습니다.일부 사용 사례는 호스팅만 필요할 수 있습니다.모델이 다른 서비스에서 Amazon SageMaker를 기존하기 전에 교육을 받았을 수도 있습니다.

이 노트북은 Amazon SageMaker XgBoost 알고리즘 컨테이너와 함께 기존의 scikit-learn을 학습한 XGBoost 모델을 사용하여 해당 모델에 대한 호스팅된 엔드포인트를 신속하게 생성하는 방법을 보여줍니다.Scikit-learn XgBoost 모델은 SageMaker XgBoost 컨테이너와 호환되지만 다른 그라데이션 부스트 트리 모델 (예: SparkML에서 훈련 된 모델) 은 그렇지 않습니다.

---
## 설정

다음을 지정하여 시작해 보겠습니다:

* AWS 리전
* 데이터에 대한 학습 및 호스팅 액세스를 제공하는 데 사용되는 IAM 역할 arn입니다.이를 지정하는 방법은 설명서를 참조하십시오.
* 학습 및 모델 데이터에 사용할 S3 버킷.

In [1]:
%%time

import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()

CPU times: user 1.38 s, sys: 1.06 s, total: 2.44 s
Wall time: 6.08 s


In [18]:
bucket

'sagemaker-ap-northeast-2-806174985048'

In [19]:
# 데이터를 저장할 버킷 지정
prefix = 'sagemaker/DEMO-xgboost-byom'
bucket_uri = 'https://s3-{}.amazonaws.com/{}'.format(region, bucket)

In [3]:
import xgboost

ModuleNotFoundError: No module named 'xgboost'

### XGboost 설치하기
conda 기반 설치의 경우 노트북 커널을 conda 및 Python3가 있는 환경으로 변경해야 합니다. 

In [4]:
!conda install -y -c conda-forge xgboost==0.90

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::imageio==2.9.0=py_0
  - conda-forge/linux-64::jupyter_server==1.4.1=py36h5fab9bb_0
  - conda-forge/noarch::black==20.8b1=py_1
  - conda-forge/linux-64::bokeh==2.2.3=py36h5fab9bb_0
  - defaults/linux-64::_anaconda_depends==5.1.0=py36_2
  - conda-forge/noarch::pyls-black==0.4.6=pyh9f0ad1d_0
  - conda-forge/noarch::aiobotocore==1.2.1=pyhd8ed1ab_0
  - conda-forge/noarch::pyls-spyder==0.3.2=pyhd8ed1ab_0
  - conda-forge/linux-64::anyio==2.1.0=py36h5fab9bb_0
  - conda-forge/noarch::jupyterlab_server==2.3.0=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib-base==3.3.4=py36hd391965_0
  - conda-forge/linux-64::spyder==4.2.0=py36h5fab9bb_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/noarch::seaborn-ba

### 데이터 집합 가져오기

In [10]:
%%time
import pickle, gzip, numpy, json

# Load the dataset
f = gzip.open('mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
f.close()

CPU times: user 671 ms, sys: 321 ms, total: 992 ms
Wall time: 991 ms


### 교육을 위한 데이터 세트 준비

In [11]:
%%time

import struct
import io
import boto3

def get_dataset():
  import pickle
  import gzip
  with gzip.open('mnist.pkl.gz', 'rb') as f:
      u = pickle._Unpickler(f)
      u.encoding = 'latin1'
      return u.load()

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 28.1 µs


In [12]:
train_set, valid_set, test_set = get_dataset()

train_X = train_set[0]
train_y = train_set[1]

valid_X = valid_set[0]
valid_y = valid_set[1]

test_X = test_set[0]
test_y = test_set[1]

In [13]:
test_y

array([7, 2, 1, ..., 4, 5, 6])

### XGBoost model 학습

In [14]:
import xgboost as xgb
import sklearn as sk 

bt = xgb.XGBClassifier(max_depth=5,
                       learning_rate=0.2,
                       n_estimators=10,
                       objective='multi:softmax')   # Setup xgboost model
bt.fit(train_X, train_y, # Train it to our data
       eval_set=[(valid_X, valid_y)], 
       verbose=False)

XGBClassifier(learning_rate=0.2, max_depth=5, n_estimators=10,
              objective='multi:softprob')

### 훈련된 모델 파일 저장

모델 파일 이름은`^ [A-ZA-Z0-9] (-* [A-ZA-Z0-9]) *; `와 같은 정규 표현식 패턴을 충족해야합니다.모델 파일도 tar-zip으로 압축해야 합니다. 

In [15]:
model_file_name = "local-xgboost-model"
bt._Booster.save_model(model_file_name)

In [16]:
!tar czvf model.tar.gz $model_file_name

local-xgboost-model


## 미리 학습된 모델을 S3에 업로드

In [20]:
fObj = open("model.tar.gz", 'rb')
key= os.path.join(prefix, model_file_name, 'model.tar.gz')
boto3.Session().resource('s3').Bucket(bucket).Object(key).upload_fileobj(fObj)

## 모델에 대한 호스팅 설정

### 호스팅 모델로 가져오기
여기에는 이전에 S3에 업로드한 모델 파일에서 SageMaker 모델을 생성하는 작업이 포함됩니다.

In [21]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost', 'latest')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [22]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://s3-{}.amazonaws.com/{}/{}'.format(region,bucket,key)
sm_client = boto3.client('sagemaker')

print (model_url)

primary_container = {
    'Image': container,
    'ModelDataUrl': model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response2['ModelArn'])

https://s3-ap-northeast-2.amazonaws.com/sagemaker-ap-northeast-2-806174985048/sagemaker/DEMO-xgboost-byom/local-xgboost-model/model.tar.gz
arn:aws:sagemaker:ap-northeast-2:806174985048:model/local-xgboost-model2021-04-05-16-58-40
CPU times: user 47.3 ms, sys: 8.01 ms, total: 55.3 ms
Wall time: 406 ms


### Endpoint 구성 만들기

SageMaker는 A/B 테스트 목적과 같이 여러 모델로 호스팅할 때 REST Endpoint 를 구성할 수 있도록 지원합니다. 이를 지원하기 위해 분할, 섀도 또는 샘플링된 방식으로 모델 간 트래픽 분포를 설명하는 Endpoint 구성을 만들 수 있습니다. 또한 엔드포인트 구성은 모델 배포에 필요한 인스턴스 유형을 설명합니다.

In [23]:
from time import gmtime, strftime

endpoint_config_name = 'DEMO-XGBoostEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'InitialVariantWeight':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

DEMO-XGBoostEndpointConfig-2021-04-05-17-08-33
Endpoint Config Arn: arn:aws:sagemaker:ap-northeast-2:806174985048:endpoint-config/demo-xgboostendpointconfig-2021-04-05-17-08-33


### endpoint 만들기

마지막으로 위에 정의된 이름과 구성을 지정하여 모델을 제공하는 endpoint 를 만듭니다. 최종 결과는 유효성을 검사하고 프로덕션 애플리케이션에 통합할 수 있는 Endpoint 입니다.이 작업을 완료하는 데 9-11분이 걸립니다.

%%time
import time

endpoint_name = 'DEMO-XGBoostEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Status: " + status)

while status=='Creating':
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp['EndpointStatus']
    print("Status: " + status)

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

## 사용할 모델의 유효성 검사

이제 이전 작업의 결과를 사용하여 클라이언트 라이브러리에서 endpoint 를 가져오고 해당 endpoint 를 사용하여 모델에서 classification 을 생성할 수 있습니다.

In [25]:
runtime_client = boto3.client('runtime.sagemaker')

단일 데이터 포인트에 대한 예측을 생성 할 수 있습니다. 이전에 생성된 테스트 데이터에서 하나를 선택합니다.

In [26]:
import numpy as np
point_X = test_X[0]
point_X = np.expand_dims(point_X, axis=0)
point_y = test_y[0]
np.savetxt("test_point.csv", point_X, delimiter=",")

In [29]:
point_X

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [27]:
%%time
import json


file_name = 'test_point.csv' #customize to your test file, will be 'mnist.single.test' if use data above

with open(file_name, 'r') as f:
    payload = f.read().strip()

response = runtime_client.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=payload)
result = response['Body'].read().decode('ascii')
print('Predicted Class Probabilities: {}.'.format(result))

Predicted Class Probabilities: [0.023023370653390884, 0.016221631318330765, 0.020377131178975105, 0.025556255131959915, 0.017341898754239082, 0.024211231619119644, 0.017396802082657814, 0.8126810789108276, 0.017022203654050827, 0.026168374344706535].
CPU times: user 10.7 ms, sys: 0 ns, total: 10.7 ms
Wall time: 170 ms


### 출력 후 처리
결과는 문자열이므로 출력 클래스 레이블을 결정하기 위해 처리

In [28]:
floatArr = np.array(json.loads(result))
predictedLabel = np.argmax(floatArr)
print('Predicted Class Label: {}.'.format(predictedLabel))
print('Actual Class Label: {}.'.format(point_y))

Predicted Class Label: 7.
Actual Class Label: 7.


### endpoint 삭제

모든 과정이 끝났으면 아래 셀에서 delete_endpoint 줄을 실행하십시오.이렇게 하면 생성한 호스팅된 endpoint 는 제거되고 더 이상 인스턴스의 요금이 부과되지 않습니다.

In [30]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '8314598e-600d-452a-b21e-72f75d405414',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8314598e-600d-452a-b21e-72f75d405414',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Mon, 05 Apr 2021 17:20:43 GMT'},
  'RetryAttempts': 0}}